<a href="https://colab.research.google.com/github/aekanun2020/2022-PEA-ML-sklearn-Spark-/blob/main/K_sckitlearn_Regression_Pipeline_Estimation_of_Arrival_Delay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import pandas as pd

In [36]:
! wget https://s3.amazonaws.com/imcbucket/data/flights/2008.csv

--2022-11-08 21:40:20--  https://s3.amazonaws.com/imcbucket/data/flights/2008.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.203.176
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.203.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689413344 (657M) [binary/octet-stream]
Saving to: ‘2008.csv.1’

2008.csv.1          100%[===================>] 657.48M  44.0MB/s    in 16s     

2022-11-08 21:40:36 (41.7 MB/s) - ‘2008.csv.1’ saved [689413344/689413344]



In [37]:
! wc -l 2008.csv

7009729 2008.csv


In [38]:
f = "2008.csv"
num_lines = sum(1 for l in open(f))
size = int(num_lines / 100)
import random
skip_idx = random.sample(range(1, num_lines), num_lines - size)
raw_pd = pd.read_csv(f, skiprows=skip_idx)

In [39]:
#raw_pd = pd.read_csv("2008.csv")

In [40]:
raw_pd.head(5)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,1229.0,1220,1342.0,1325,WN,2874,...,2.0,24.0,0,NaN,0,9.0,0.0,8.0,0.0,0.0
1,2008,1,3,4,1424.0,1415,1941.0,1945,WN,396,...,8.0,12.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,934.0,935,1453.0,1505,WN,830,...,6.0,16.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,851.0,835,938.0,930,WN,1670,...,5.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1503.0,1425,1604.0,1530,WN,2357,...,3.0,10.0,0,NaN,0,9.0,0.0,0.0,0.0,25.0


In [41]:
raw_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70096 entries, 0 to 70095
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Year               70096 non-null  int64  
 1   Month              70096 non-null  int64  
 2   DayofMonth         70096 non-null  int64  
 3   DayOfWeek          70096 non-null  int64  
 4   DepTime            68772 non-null  float64
 5   CRSDepTime         70096 non-null  int64  
 6   ArrTime            68624 non-null  float64
 7   CRSArrTime         70096 non-null  int64  
 8   UniqueCarrier      70096 non-null  object 
 9   FlightNum          70096 non-null  int64  
 10  TailNum            69259 non-null  object 
 11  ActualElapsedTime  68588 non-null  float64
 12  CRSElapsedTime     70090 non-null  float64
 13  AirTime            68588 non-null  float64
 14  ArrDelay           68588 non-null  float64
 15  DepDelay           68772 non-null  float64
 16  Origin             700

In [42]:
def t_timeperiod(origin):
    if origin is None:
        period = None
    elif origin > 0 and origin < 600:
        period = '00.01-05.59'
    elif origin >= 600 and origin <=1200:
        period = '06.00-11.59'
    elif origin >= 1200 and origin <= 1800:
        period = '12.00-17.59'
    elif origin >= 1800 and origin <= 2400:
        period = '18.00-24.00'
    else:
        period = 'NA'
    return period

In [43]:
raw_pd['DepTime'] = raw_pd['DepTime'].apply(lambda x:t_timeperiod(x))

In [44]:
max_distance = raw_pd['Distance'].max()
min_distance = raw_pd['Distance'].min()

In [45]:
max_ArrDelay = raw_pd['ArrDelay'].max()
min_ArrDelay = raw_pd['ArrDelay'].min()

In [46]:
def t_normalized_distance(origin):
    if origin is None:
        return None
    else:
        return ((origin-min_distance)/(max_distance-min_distance))

In [47]:
def t_normalized_ArrDelay(origin):
    if origin is None:
        return None
    else:
        return ((origin-min_ArrDelay)/(max_ArrDelay-min_ArrDelay))

In [48]:
raw_pd['Distance'] = raw_pd['Distance'].apply(lambda x:t_normalized_distance(x))

In [49]:
raw_pd['ArrDelay'] = raw_pd['ArrDelay'].apply(lambda x:t_normalized_ArrDelay(x))

In [50]:
features_pd = raw_pd[['UniqueCarrier','Origin','Dest',\
        'DepTime','TaxiOut','TaxiIn','DepDelay',\
        'DayOfWeek','Distance','ArrDelay']]

In [51]:
final_pd = features_pd.dropna()

In [52]:
final_pd.count()

UniqueCarrier    68588
Origin           68588
Dest             68588
DepTime          68588
TaxiOut          68588
TaxiIn           68588
DepDelay         68588
DayOfWeek        68588
Distance         68588
ArrDelay         68588
dtype: int64

In [53]:
features_pd.head(20)

,UniqueCarrier,Origin,Dest,DepTime,TaxiOut,TaxiIn,DepDelay,DayOfWeek,Distance,ArrDelay
0,WN,LAS,BUR,12.00-17.59,24.0,2.0,9.0,4,0.038937,0.082765
1,WN,LAS,MDW,12.00-17.59,12.0,8.0,9.0,4,0.302170,0.064846
2,WN,LAS,MDW,06.00-11.59,16.0,6.0,-1.0,4,0.302170,0.058020
3,WN,LAS,ONT,06.00-11.59,8.0,5.0,16.0,4,0.033665,0.075085
4,WN,LAS,SAN,12.00-17.59,10.0,3.0,38.0,4,0.046035,0.097270
5,WN,LAS,TUS,12.00-17.59,17.0,3.0,21.0,4,0.067735,0.086177
6,WN,LAX,AUS,06.00-11.59,7.0,5.0,9.0,4,0.245589,0.067406
7,WN,MCO,BUF,06.00-11.59,12.0,3.0,6.0,4,0.198743,0.079352
8,WN,MCO,PVD,06.00-11.59,13.0,4.0,2.0,4,0.211316,0.073379
9,WN,MCO,RSW,18.00-24.00,16.0,3.0,8.0,4,0.020685,0.076792


In [54]:


#raw_pd.isnull().sum()
#nullCol_series = raw_pd.isnull().sum()
#notNull_list = list(nullCol_series[nullCol_series == 0].index)
#notNull_list
#requiredCol_list = ['TaxiIn','TaxiOut','ArrDelay']
#selectedCol_list = notNull_list + requiredCol_list
#noNull_pd = raw_pd.loc[:,selectedCol_list].dropna()
#noNull_pd.isnull().sum()
#noNull_pd.describe()
#noNull_pd.info()

In [55]:
#final_pd = noNull_pd

In [56]:
import pandas as pd
from seaborn import load_dataset
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

In [57]:
train_df = final_pd.sample(frac=0.8,random_state=123)

In [58]:
test_df = final_pd.drop(train_df.index)

In [59]:
train_df.count()

UniqueCarrier    54870
Origin           54870
Dest             54870
DepTime          54870
TaxiOut          54870
TaxiIn           54870
DepDelay         54870
DayOfWeek        54870
Distance         54870
ArrDelay         54870
dtype: int64

In [60]:
test_df.count()

UniqueCarrier    13718
Origin           13718
Dest             13718
DepTime          13718
TaxiOut          13718
TaxiIn           13718
DepDelay         13718
DayOfWeek        13718
Distance         13718
ArrDelay         13718
dtype: int64

In [61]:
train_labels = train_df.filter(['ArrDelay'])
train_df = train_df.loc[:, train_df.columns != 'ArrDelay']

In [62]:
type(train_labels)

pandas.core.frame.DataFrame

In [63]:
test_labels = test_df.filter(['ArrDelay'])
test_df = test_df.loc[:, test_df.columns != 'ArrDelay']

In [64]:
X_train = train_df
X_test = test_df
y_train = train_labels
y_test = test_labels

In [65]:
categorical = list(X_train.select_dtypes('object').columns)
print(f"Categorical columns are: {categorical}")

numerical = list(X_train.select_dtypes('number').columns)
print(f"Numerical columns are: {numerical}")

Categorical columns are: ['UniqueCarrier', 'Origin', 'Dest', 'DepTime']
Numerical columns are: ['TaxiOut', 'TaxiIn', 'DepDelay', 'DayOfWeek', 'Distance']


In [66]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz, DecisionTreeRegressor
from sklearn import tree
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [67]:
# Define custom transformer
class ColumnSelector(BaseEstimator, TransformerMixin):
    """Select only specified columns."""
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.columns]

# Define categorical pipeline
cat_pipe = Pipeline([('selector', ColumnSelector(categorical)),
                     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))])

# Define numerical pipeline
num_pipe = Pipeline([('selector', ColumnSelector(numerical))
                     #('imputer', SimpleImputer(strategy='median')),
                     #('scaler', MinMaxScaler())
                    ])

# Combine categorical and numerical pipeline
preprocessor = FeatureUnion(transformer_list=[('cat', cat_pipe),
                                              ('num', num_pipe)])

# Model Training
pipe = Pipeline(steps=[('preprocessor', preprocessor),
              ('model', DecisionTreeRegressor(max_depth=None))])
pipeline_model = pipe.fit(X_train, y_train)

In [68]:
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

predictions = pipeline_model.predict(X_test)
print("\t RMSE:", mean_squared_error(y_test, predictions, squared=False))
print("\t R2 score:", r2_score(y_test, predictions))

	 RMSE: 0.011311086869999798
	 R2 score: 0.8809247022629536
